## Raytracing Off-Axis Parabola Trains

In this tutorial, we will show how to draw a system made of off-axis parabola relays.  These are an area in which prysm is substantially different to lens design programs.

We begin, as in the [fundamental tutorial](./Raytracing-Fundamentals.ipynb), with a bunch of imports

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

from prysm.experimental.raytracing.surfaces import Surface
from prysm.experimental.raytracing.spencer_and_murty import raytrace
from prysm.experimental.raytracing.raygen import generate_collimated_ray_fan
from prysm.experimental.raytracing.plotting import plot_rays, plot_optics

from matplotlib import pyplot as plt

Before actually laying out a prescription, first we should review some simple truths about an off-axis parabola.  Firstly, OAPs may be defined in several ways; the angle between the gut axis and the axis of rotation of the parent, the off-axis distance, measured to the "lower" mechanical edge of the part, the off-axis distance measured to the segment vertex, or the off-axis distanced measured to another datum.

prysm uses essentially the third option.  The off-axis distance is a shift of the coordinate system, which if the OAP's mechanical aperture is symmetric, is the mechanical center of the off-axis segment.

There is no way to specify an OAP by its off-axis angle.

Secondly, any two OAPs which have the same off-axis distance form a stigmatic pair for an input object at infinity.

Thirdly, changing the radius of curvature (or parent focal length) of the OAP gives the relay any magnification.

With this in mind, we'll lay out a simple unit magnification periscope:

In [ ]:
mirror_semidiameter=25
c=-0.002
parent_focal_length = 1/c/2
dy=35
k=-1
pres = [
    Surface.off_axis_conic(c, k, 'refl', -parent_focal_length, dy=dy),
    Surface.off_axis_conic(-c, k, 'refl', parent_focal_length, dy=dy),
    Surface.plane('eval', -parent_focal_length)
]

P, S = generate_collimated_ray_fan(nrays=8, maxr=mirror_semidiameter, z=parent_focal_length)
phist, shist = raytrace(pres, P, S, 0.6328)

fig, ax = plt.subplots(figsize=(15,15))
fig, ax = plot_rays(phist, fig=fig, ax=ax)
ax.axhline(-dy, ls=':', c='#aaa')
plot_optics(pres, phist, lw=2, fig=fig, ax=ax)
ax.set(aspect='equal')

Note that this parametric layout is only so trivial because the focus is intentionally plced at the Z-X origin.  For a sequence of relays, simply book-keep where you want the focus between the OAPs to be and the same triviality is maintained.

The sign of dy is the same for each because the sign of the curvatures is opposite.  If we adjust the curvature of either OAP, we can make a beam compressor (or expander):

In [ ]:
mirror_semidiameter=25
c=-0.002
parent_focal_length = 1/c/2
dy=35
k=-1
m = 0.2 # magnification
pres = [
    Surface.off_axis_conic(c, k, 'refl', -parent_focal_length, dy=dy),
    Surface.off_axis_conic(-c*m, k, 'refl', parent_focal_length/m, dy=dy),
    Surface.plane('eval', -parent_focal_length)
]

P, S = generate_collimated_ray_fan(nrays=8, maxr=mirror_semidiameter, z=parent_focal_length/m)
phist, shist = raytrace(pres, P, S, 0.6328)

fig, ax = plt.subplots(figsize=(15,15))
fig, ax = plot_rays(phist, fig=fig, ax=ax)
ax.axhline(-dy, ls=':', c='#aaa')
plot_optics(pres, phist, lw=2, fig=fig, ax=ax)
ax.set(aspect='equal')

We'll add a fold and another OAP to the first design.  First, we'll just use the fold mirror and a plane normal to Z to see where the rays go.  Unfortunately, we can't use plot_optics anymore, since it doesn't yet understand tilted surfaces (fear not, the raytracer does).

In [ ]:
mirror_semidiameter=25
c=-0.002
parent_focal_length = 1/c/2
dy=35
k=-1
pres = [
    Surface.off_axis_conic(c, k, 'refl', -parent_focal_length, dy=dy),
    Surface.off_axis_conic(-c, k, 'refl', parent_focal_length, dy=dy),
    Surface.plane('refl', -parent_focal_length, R=(0,-8,0)),
    Surface.plane('refl', parent_focal_length)
]

P, S = generate_collimated_ray_fan(nrays=8, maxr=mirror_semidiameter, z=parent_focal_length)
phist, shist = raytrace(pres, P, S, 0.6328)

fig, ax = plt.subplots(figsize=(15,15))
fig, ax = plot_rays(phist, fig=fig, ax=ax)
ax.axhline(-dy, ls=':', c='#aaa')
# plot_optics(pres, phist, lw=2, fig=fig, ax=ax)
ax.set(aspect='equal')

Another fold mirror for fun,

In [ ]:
# ray starting from 0, since OAP1 is centered on X and Y=0, and going in the +Z direction
mirror_semidiameter=25
c=-0.002
parent_focal_length = 1/c/2
dy=35
k=-1
pres2 = [
    Surface.off_axis_conic(c, k, 'refl', -parent_focal_length, dy=dy),
    Surface.off_axis_conic(-c, k, 'refl', parent_focal_length, dy=dy),
    Surface.plane('refl', -parent_focal_length, R=(0,-8,0)),
    Surface.plane('refl', parent_focal_length, R=(0,-8,0)),
    Surface.plane('eval', -parent_focal_length)
]

P, S = generate_collimated_ray_fan(nrays=8, maxr=mirror_semidiameter, z=parent_focal_length)
phist, shist = raytrace(pres2, P, S, 0.6328)

fig, ax = plt.subplots(figsize=(15,15))
fig, ax = plot_rays(phist, fig=fig, ax=ax)
ax.axhline(-dy, ls=':', c='#aaa')

We can place a final OAP, not centered on y=0, which focuses the beam sheared to the input port,

In [ ]:
# ray starting from 0, since OAP1 is centered on X and Y=0, and going in the +Z direction
mirror_semidiameter=25
c=-0.002
parent_focal_length = 1/c/2
dy=35
k=-1
pres3 = [
    Surface.off_axis_conic(c, k, 'refl', -parent_focal_length, dy=dy),
    Surface.off_axis_conic(-c, k, 'refl', parent_focal_length, dy=dy),
    Surface.plane('refl', -parent_focal_length, R=(0,-8,0)),
    Surface.plane('refl', parent_focal_length, R=(0,-8,0)),
    Surface.plane('refl', -parent_focal_length),
    # give two elements for P = Y, Z
    Surface.off_axis_conic(c/2, k, 'refl', [2*dy, -parent_focal_length], dy=-50),
    Surface.plane('eval', parent_focal_length)
]

P, S = generate_collimated_ray_fan(nrays=8, maxr=mirror_semidiameter, z=parent_focal_length)
phist, shist = raytrace(pres3, P, S, 0.6328)

fig, ax = plt.subplots(figsize=(15,15))
fig, ax = plot_rays(phist, fig=fig, ax=ax)
ax.axhline(-dy, ls=':', c='#aaa')
ax.set(aspect='equal')

and like so, we have a fairly complicated layout with three OAPs and two fold mirrors.

## Wrap-Up

To raytrace a series of OAPs, describe them and any fold mirrors in global coordinates.  Keep the properties of OAPs in mind if you are not replicating an existing design.  If you are replicating an existing design, I recommend having the designer export a chief ray trace and the position and direction cosines all the way through, so that you can cross-verify that the prescription you lay out in prysm is equivalent.